# Health Insurance Recommendation System with CrewAI

This notebook uses **CrewAI** with **Ollama (llama3.2)** to provide personalized health insurance recommendations.

## System Architecture
- **User Profiler Agent**: Analyzes user inputs and creates a detailed profile
- **Recommendation Agent**: Matches user profile against all plans and ranks them
- **Comparison Agent**: Creates detailed comparison with reasoning

In [ ]:
# Import required libraries
import json
import pandas as pd
import os
from crewai import Agent, Task, Crew, Process
import warnings
warnings.filterwarnings('ignore')

# Configure Ollama via environment variables (this is the fix!)
os.environ["OPENAI_API_BASE"] = "http://localhost:11434/v1"
os.environ["OPENAI_MODEL_NAME"] = "llama3.2"
os.environ["OPENAI_API_KEY"] = "ollama"  # Dummy key for local Ollama

print("✅ Libraries imported and Ollama configured via environment!")

In [ ]:
# Load insurance data
with open('data/indian_health_insurance_data.json', 'r') as f:
    insurance_data = json.load(f)

# Extract insurers and plans
insurers = insurance_data['insurers']
total_plans = sum(len(insurer['plans']) for insurer in insurers)

print(f"✅ Loaded data for {len(insurers)} insurers")
print(f"✅ Total plans available: {total_plans}")
print(f"\nInsurers: {', '.join([i['insurer_name'] for i in insurers])}")

In [ ]:
# Define CrewAI Agents
# NOTE: We DO NOT pass llm parameter - agents use environment variables automatically

# Agent 1: User Profiler
user_profiler = Agent(
    role='Health Insurance Needs Analyst',
    goal='Analyze user requirements and create a comprehensive profile of their insurance needs',
    backstory="""You are an expert health insurance analyst who specializes in understanding 
    customer needs. You excel at identifying the most important factors for each customer based 
    on their age, health conditions, budget, and preferences.""",
    verbose=True,
    allow_delegation=False
)

# Agent 2: Recommendation Agent
recommendation_agent = Agent(
    role='Insurance Plan Matcher',
    goal='Analyze all available insurance plans and rank the top 3 best matches for the user profile',
    backstory="""You are a highly experienced insurance plan matcher with deep knowledge of 
    Indian health insurance products. You excel at analyzing plan features like waiting periods, 
    coverage limits, claim settlement ratios, and special benefits.""",
    verbose=True,
    allow_delegation=False
)

# Agent 3: Comparison Agent
comparison_agent = Agent(
    role='Insurance Comparison Specialist',
    goal='Create a detailed comparison of recommended plans with clear reasoning for rankings',
    backstory="""You are an insurance comparison expert who excels at breaking down complex 
    insurance features into simple, understandable comparisons. You provide clear reasoning for 
    why certain plans are better suited for specific customer needs.""",
    verbose=True,
    allow_delegation=False
)

print("✅ Created 3 specialized agents")

In [ ]:
# User Input Collection
print("=" * 60)
print("HEALTH INSURANCE RECOMMENDATION SYSTEM")
print("=" * 60)
print("\nPlease provide your details for personalized recommendations:\n")

user_age = input("1. Your Age: ")
user_ped = input("2. Pre-existing Conditions (comma-separated, or 'None'): ")
user_budget = input("3. Budget Range (e.g., '15000-25000' per year): ")
user_needs = input("4. Specific Needs (e.g., maternity, OPD, no room rent limit): ")
user_preferences = input("5. Preferred Features (e.g., wellness rewards, quick claim settlement): ")

# Create user profile dictionary
user_profile = {
    'age': user_age,
    'pre_existing_conditions': user_ped,
    'budget': user_budget,
    'specific_needs': user_needs,
    'preferences': user_preferences
}

print("\n✅ User profile collected successfully!")
print(f"\nProfile Summary:")
for key, value in user_profile.items():
    print(f"  - {key.replace('_', ' ').title()}: {value}")

In [ ]:
# Create insurance context (simplified version)
def create_insurance_context():
    """Create a concise context string with all insurance plan details"""
    context = "AVAILABLE HEALTH INSURANCE PLANS:\n\n"
    
    for insurer in insurers:
        context += f"\n{'='*50}\n"
        context += f"INSURER: {insurer['insurer_name']}\n"
        context += f"CSR: {insurer['claim_settlement_ratio']['csr_percentage']}\n"
        
        for plan in insurer['plans']:
            context += f"\n--- {plan['plan_name']} ---\n"
            coverage = plan['coverage_static']
            waiting = plan['waiting_periods']
            
            context += f"Sum Insured: {coverage.get('sum_insured_options', 'N/A')}\n"
            context += f"Room Rent: {coverage.get('room_rent_limit', 'N/A')}\n"
            context += f"NCB: {coverage.get('no_claim_bonus', 'N/A')}\n"
            context += f"Maternity: {coverage.get('maternity_coverage', 'N/A')}\n"
            context += f"OPD: {coverage.get('opd_coverage', 'N/A')}\n"
            context += f"PED Waiting: {waiting.get('ped_waiting_months', 'N/A')} months\n"
            context += f"Key Features: {', '.join(plan.get('key_features', [])[:3])}\n"
    
    return context

insurance_context = create_insurance_context()
print(f"✅ Created insurance context")

In [ ]:
# Define CrewAI Tasks

# Task 1: Profile the user
profile_task = Task(
    description=f"""Analyze this user and list their key insurance needs:
    Age: {user_profile['age']}, PED: {user_profile['pre_existing_conditions']}, 
    Budget: {user_profile['budget']}, Needs: {user_profile['specific_needs']}""",
    agent=user_profiler,
    expected_output="User needs profile with priorities"
)

# Task 2: Recommend plans
recommend_task = Task(
    description=f"""Based on user needs, recommend TOP 3 insurance plans from:
    {insurance_context[:2000]}
    
    Output format: Plan Name | Insurer | Why Recommended""",
    agent=recommendation_agent,
    expected_output="Top 3 ranked plans with reasoning",
    context=[profile_task]
)

# Task 3: Create comparison
compare_task = Task(
    description="""Create a comparison table of the 3 recommended plans with brief reasoning.""",
    agent=comparison_agent,
    expected_output="Comparison table with reasoning",
    context=[profile_task, recommend_task]
)

print("✅ Created 3 tasks")

In [ ]:
# Create and Run the Crew
print("\n" + "="*60)
print("STARTING HEALTH INSURANCE RECOMMENDATION ANALYSIS")
print("="*60)
print("\nThis may take 2-3 minutes as agents analyze all plans...\n")

crew = Crew(
    agents=[user_profiler, recommendation_agent, comparison_agent],
    tasks=[profile_task, recommend_task, compare_task],
    process=Process.sequential,
    verbose=True
)

# Execute the crew
result = crew.kickoff()

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)

In [ ]:
# Display the results
print("\n📊 RECOMMENDATION RESULTS:\n")
print(result)
print("\n✅ Recommendation complete!")

## Demo Examples

Below are some pre-configured examples you can try:

In [ ]:
# Example User Profiles for Testing

demo_profiles = {
    'young_professional': {
        'age': '28',
        'pre_existing_conditions': 'None',
        'budget': '10000-15000',
        'specific_needs': 'No room rent limit, good claim settlement',
        'preferences': 'Wellness rewards, digital experience'
    },
    'family_with_kids': {
        'age': '35',
        'pre_existing_conditions': 'None',
        'budget': '20000-30000',
        'specific_needs': 'Maternity coverage, family floater, OPD',
        'preferences': 'Large hospital network, quick claim settlement'
    },
    'senior_citizen': {
        'age': '62',
        'pre_existing_conditions': 'Diabetes, Hypertension',
        'budget': '30000-50000',
        'specific_needs': 'Short PED waiting, no co-payment, home care',
        'preferences': 'Premium features, excellent CSR, restoration benefit'
    }
}

print("Demo Profiles Available:")
for name, profile in demo_profiles.items():
    print(f"\n{name.replace('_', ' ').title()}:")
    for key, value in profile.items():
        print(f"  - {key.replace('_', ' ').title()}: {value}")

print("\n\nTo use a demo profile, copy the values to the input cell above.")